# ST 590 Homework 6 By: Eric Warren

## Part 1: Split Data

Using the NFL box score data set ([https://www4.stat.ncsu.edu/~online/datasets/scoresFull.csv](https://www4.stat.ncsu.edu/~online/datasets/scoresFull.csv)), we are going to split the data into separate .csv files based on the season. That is, you want to subset the data to obtain just one season and output that to a .csv file. You want to do this process for each season in the dataset.

In [54]:
# Import pandas
import pandas as pd

# Read in the csv file
nfl_full = pd.read_csv("https://www4.stat.ncsu.edu/~online/datasets/scoresFull.csv")

# Split the data into new csvs
for i, value in enumerate(nfl_full['season']):
        nfl_full[nfl_full['season'] == value].to_csv(r'NFL_season_'+str(value)+r'.csv',index = False, na_rep = 'N/A')

## Part 2: MapReduce Idea (no pyspark)

### MapReduce Part

- Consider a variable to group on (in our case we are going to use `week`) and a numeric target variable (in our case we are going to use `AQ1`)
- Now we are going to write a mapping function to find the
  - Sum of the target variable across the grouping variable
  - Sum of the squared values of the target variable across the grouping variable
  - Count or number of observations of the target variable in each group
- Lastly, write a reduce function to combine the results across the season data sets

In [55]:
# Import reduce function
from functools import reduce

# Create mapping function
def find_sums(file, grouping_var = "week", numeric_var = "AQ1"):
  """
  We are first going to read in our data based on the file given
  In this function we are going to find the sum of the target variable across the grouping variable
  We are also going to find the sum of squared values across the grouping variable
  Lastly, we should return the count of the variable for each of its groups
  """

  # Read in the data
  data = pd.read_csv(file)

  # Get the sum of a value
  sum = data.groupby(by = [grouping_var], as_index = False)[numeric_var].sum()

  # Get the sum of squared values
  sum_sq = data[numeric_var].pow(2).groupby(data[grouping_var]).sum()

  # Get the count present
  count = data.groupby(by = [grouping_var], as_index = False).size()

  # Combine the data together
  values = reduce(lambda x, y: pd.merge(x, y, on = grouping_var), [sum, sum_sq, count])

  # Get the season
  values['season'] = file[-8:-4]

  # Rename some columns to make it easier to follow
  values = values.rename(columns={numeric_var + "_x" : "sum", numeric_var + "_y": "sum_sq"})
  return values

# Now create list to get the different csv files
lst_seasons = []
for szn in range(nfl_full.season.min(), nfl_full.season.max() + 1):
  lst_seasons.append("NFL_season_" + str(szn) + ".csv")

# Now get the values for each season to show that this can work
results = list(map(lambda x: find_sums(x), lst_seasons)) # This gives list of dfs
results_df = pd.concat(results) # This combines list of dfs into a larger df
results_df

,week,sum,sum_sq,size,season
0,1,81,695,16,2002
1,10,78,686,14,2002
2,11,52,450,16,2002
3,12,68,666,16,2002
4,13,56,534,16,2002
...,...,...,...,...,...
16,9,36,192,13,2014
17,ConfChamp,13,169,2,2014
18,Division,21,245,4,2014
19,SuperBowl,0,0,1,2014


### Summarizing Bit

- Now take the final result and use it to construct the
  - mean at each level of the grouping variable
  - standard deviation at each level of the grouping variable (if the count for the group was larger than 1)
- We are going to show our function works for the 2002 season

In [56]:
# Import numpy to use where function
import numpy as np

# Create a function to find the mean and sd
def find_mean_sd(data, grouping_var = "week"):
  """
  Here we would like to find the mean and standard deviation at each level of grouping variable
  """
  # First get a sum of the sum of data for numeric column
  sum = data.groupby(by = [grouping_var], as_index = False)['sum'].sum()

  # Now get a sum of the sum of squares of data for numeric column
  sum_sq = data.groupby(by = [grouping_var], as_index = False)['sum_sq'].sum()

  # Now get a sum of the count of data for numeric column
  size = data.groupby(by = [grouping_var], as_index = False)['size'].sum()

  # Now combine into one file
  df = reduce(lambda x, y: pd.merge(x, y, on = grouping_var), [sum, sum_sq, size])

  # Get the mean by group
  df['mean'] = df['sum'] / df['size']

  # Now get the standard deviation
  df['sd'] = np.where(df['size'] == 1, 
                      None, 
                      np.sqrt((1 / (df['size'] - 1)) * (df['sum_sq'] - (df['mean'] ** 2 * df['size']))))

  return df[['week', 'mean', 'sd']]

# Show this works on data
test = find_sums("NFL_season_2002.csv") # 2002 season
find_mean_sd(test)

,week,mean,sd
0,1,5.062500,4.358421
1,10,5.571429,4.397802
2,11,3.250000,4.328202
3,12,4.250000,5.013316
4,13,3.500000,4.746929
5,14,5.500000,5.316641
6,15,3.125000,3.930649
7,16,5.750000,5.013316
8,17,3.187500,4.134711
9,2,4.187500,5.256345


Now that we can see our mean and standard deviation function works for our 2002 NFL season, we can do this to find the values for all the seasons grouped together.

Lastly, we are going to create a function to put the MapReduce part and the final calculation part into an easy to use function, allowing you to change the grouping variable and target variable.

In [57]:
def grouping_mean_sd(file, grouping_var = "week", numeric_var = "AQ1"):
  """
  Here we are going to combine our function to get sum, sum squares, and length
  into our other function to get sample mean and sample standard deviations
  from our files directly
  """
  
  # First get the data we need from the first function
  if isinstance(file, str):
    data = find_sums(file, grouping_var = grouping_var, numeric_var = numeric_var)
  else:
    results = list(map(lambda x: find_sums(x), file)) # This gives list of dfs
    data = pd.concat(results)

  # Now use other function to get the mean and standard deviation of each grouping
  final_data = find_mean_sd(data, grouping_var = grouping_var)

  # Return our final groupings 
  return final_data

We can check this works on a simple data set being our 2002 data.

In [58]:
# Show it works for our 2002 data
grouping_mean_sd("NFL_season_2002.csv")

,week,mean,sd
0,1,5.062500,4.358421
1,10,5.571429,4.397802
2,11,3.250000,4.328202
3,12,4.250000,5.013316
4,13,3.500000,4.746929
5,14,5.500000,5.316641
6,15,3.125000,3.930649
7,16,5.750000,5.013316
8,17,3.187500,4.134711
9,2,4.187500,5.256345


Now show it works for all seasons we are trying to look at (2002 - 2014)

In [59]:
# Show it works for all seasons
grouping_mean_sd(lst_seasons)

,week,mean,sd
0,1,3.427885,4.041403
1,10,4.580645,4.45111
2,11,3.970149,4.513214
3,12,3.653659,4.311641
4,13,4.192308,4.765165
5,14,3.836538,4.502518
6,15,3.913462,4.63261
7,16,4.096154,4.239267
8,17,3.725962,4.548918
9,2,3.531401,4.195204


It is interesting to see that the away team's tend to score the most first quarter point on average in the Wild Card and Divisional Rounds (or earlier rounds in the playoffs). However, the "home field" advantage we are used to with the away teams struggling tend to show up more in later rounds of the playoffs. Also note that the variability (standard deviation) tends to be larger when the mean is larger meaning some observations could have made this average higher. For example, there are less playoff games played each year (and even less in later rounds) than regular season matchups. So it is not surprising to see the extreme sampling averages show up in these rounds in particular. 

## Part 3: Using pyspark (SQL)

We'll use spark SQL functionality (rather than writing our own MapReduce type code.) To do:

- Read in the full nfl data set into spark as a spark SQL style data frame
- Use spark SQL to find the mean and standard deviation for the AQ1, AQ2, AQ3, AQ4, AFinal, HQ1, HQ2, HQ3, HQ4, and HFinal variables

In [60]:
# Make data from URL into csv saved on local
pd.read_csv("https://www4.stat.ncsu.edu/~online/datasets/scoresFull.csv").to_csv(r'NFL_data.csv',index = False, na_rep = 'N/A')

# Start up Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('Warren_HW6').getOrCreate()

# Load in data
df = spark.read.load("NFL_data.csv",
                     format="csv",
                     sep=",",
                     inferSchema="true",
                     header="true")

# Get the sum of all the data
df.select(["AQ1", "AQ2", "AQ3", "AQ4", "AFinal", "HQ1", "HQ2", "HQ3", "HQ4", "HFinal"]).groupBy().sum().show()

+--------+--------+--------+--------+-----------+--------+--------+--------+--------+-----------+
|sum(AQ1)|sum(AQ2)|sum(AQ3)|sum(AQ4)|sum(AFinal)|sum(HQ1)|sum(HQ2)|sum(HQ3)|sum(HQ4)|sum(HFinal)|
+--------+--------+--------+--------+-----------+--------+--------+--------+--------+-----------+
|   13623|   21664|   15227|   20445|      71354|   16761|   24662|   16630|   21947|      80437|
+--------+--------+--------+--------+-----------+--------+--------+--------+--------+-----------+



We can see here how the 2nd and 4th quarters for away teams seem to be the best quarters, while the same holds true for the home teams. Overall, we can see how home teams perform better than away teams by scoring points.

Repeat the previous item but return summaries at each level of the season variable.

In [61]:
# Get the sum of all the data
df.select(["season", "AQ1", "AQ2", "AQ3", "AQ4", "AFinal", "HQ1", "HQ2", "HQ3", "HQ4", "HFinal"]).groupBy("season").sum().sort("season").drop("sum(season)").show()

+------+--------+--------+--------+--------+-----------+--------+--------+--------+--------+-----------+
|season|sum(AQ1)|sum(AQ2)|sum(AQ3)|sum(AQ4)|sum(AFinal)|sum(HQ1)|sum(HQ2)|sum(HQ3)|sum(HQ4)|sum(HFinal)|
+------+--------+--------+--------+--------+-----------+--------+--------+--------+--------+-----------+
|  2002|    1078|    1608|    1188|    1583|       5511|    1150|    1979|    1259|    1716|       6146|
|  2003|     950|    1627|    1045|    1430|       5103|    1342|    1769|    1218|    1681|       6055|
|  2004|    1045|    1673|    1109|    1548|       5404|    1320|    1888|    1135|    1752|       6116|
|  2005|    1038|    1480|    1076|    1386|       5016|    1173|    1983|    1249|    1532|       5958|
|  2006|     969|    1650|    1149|    1619|       5408|    1230|    1613|    1219|    1584|       5676|
|  2007|     987|    1661|    1158|    1568|       5401|    1342|    1889|    1312|    1607|       6183|
|  2008|    1015|    1735|    1088|    1715|       5565

Here we can see the constant trend over the seasons that scoring is larger in the second and fourth quarters for both home and away teams. This is an interesting trend to note. It is also amazing to see how each year the home team scored more overall points than the away team. This might show home field advantage is real.

## Using pyspark (pandas-on-spark)

Repeat part 3 but read the data into a pandas-on-spark data frame and use pandas-on-spark functionality
to find the summaries!

- Read in the full nfl data set into spark as a pandas on spark style data frame
- Use spark SQL to find the mean and standard deviation for the AQ1, AQ2, AQ3, AQ4, AFinal, HQ1, HQ2, HQ3, HQ4, and HFinal variables

In [62]:
# Import pyspark on pandas
import pyspark.pandas as ps

# Read in the data
pdf = ps.read_csv("NFL_data.csv")

# Make the first sum without grouping
pdf[["AQ1", "AQ2", "AQ3", "AQ4", "AFinal", "HQ1", "HQ2", "HQ3", "HQ4", "HFinal"]].sum()

/opt/tljh/user/envs/pySpark/lib/python3.9/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


AQ1       13623
AQ2       21664
AQ3       15227
AQ4       20445
AFinal    71354
HQ1       16761
HQ2       24662
HQ3       16630
HQ4       21947
HFinal    80437
dtype: int64

Same trends as before from what we saw in **Part 3**. Repeat the previous item but return summaries at each level of the season variable.

In [63]:
# Make the sum with grouping on season
pdf[["season", "AQ1", "AQ2", "AQ3", "AQ4", "AFinal", "HQ1", "HQ2", "HQ3", "HQ4", "HFinal"]].groupby("season", as_index = False).sum().sort_values(by = ['season'])

/opt/tljh/user/envs/pySpark/lib/python3.9/site-packages/pyspark/pandas/groupby.py:893: FutureWarning: Default value of `numeric_only` will be changed to `False` instead of `True` in 4.0.0.
  warnings.warn(


,season,AQ1,AQ2,AQ3,AQ4,AFinal,HQ1,HQ2,HQ3,HQ4,HFinal
12,2002,1078,1608,1188,1583,5511,1150,1979,1259,1716,6146
0,2003,950,1627,1045,1430,5103,1342,1769,1218,1681,6055
5,2004,1045,1673,1109,1548,5404,1320,1888,1135,1752,6116
8,2005,1038,1480,1076,1386,5016,1173,1983,1249,1532,5958
2,2006,969,1650,1149,1619,5408,1230,1613,1219,1584,5676
1,2007,987,1661,1158,1568,5401,1342,1889,1312,1607,6183
11,2008,1015,1735,1088,1715,5565,1383,1924,1227,1614,6190
7,2009,1033,1656,1148,1587,5442,1265,2080,1129,1572,6082
9,2010,1061,1833,1236,1523,5692,1222,1808,1300,1784,6141
10,2011,1030,1597,1224,1601,5476,1341,1958,1344,1730,6403


Again we get the same trends that are mentioned in **Part 3**.